In [18]:
from scipy import stats
import scipy
import numpy as np

Давайте уточним правило трёх сигм. Утверждение: $99.7\%$ вероятностной массы случайной величины $X∼N(\mu,\sigma2)$ лежит в интервале $\mu\pm c \cdot \sigma$. Чему равно точное значение константы $c$? Округлите ответ до четырёх знаков после десятичной точки.

In [3]:
round(stats.norm.ppf(1-0.003/2),4)

2.9677

В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [11]:
#сгенерируем данные
aspirin_takers = [1 if i < 104 else 0 for i in range(11037)]
placebo_takers = [1 if i <189 else 0 for i in range(11034)]
#вычислим вероятности
aspirin_proba = round(sum(aspirin_takers)/ len(aspirin_takers),4)
placebo_proba = round(sum(placebo_takers) / len(placebo_takers),4)

#искомая вероятность является разностью вероятностей смерти
print('Снижение вероятности:',abs(aspirin_proba-placebo_proba))

Снижение вероятности: 0.0077


Постройте теперь $95 \%$ доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.



In [16]:
#берём функцию из тренировочного ноутбука
#дальше остаётся просто подставить наши значения
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [21]:
round(proportions_confint_diff_ind(placebo_takers,aspirin_takers)[1],4)

0.0107

Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин $X∼Ber(p)$ часто вычисляют величину $\frac{p}{1-p}$ , которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо $p$ нужно подставить $p$. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как

$\frac{\frac{189}{11034}}{1-\frac{189}{11034}} = \frac{189}{11034-189}\approx 0.0174 $

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [22]:
def odds(X):
    return (sum(X)/len(X))/(1-(sum(X)/len(X)))

In [26]:
print('ответ:', round(odds(placebo_takers)/odds(aspirin_takers),4))

ответ: 1.8321


Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;

установите random seed=0;


сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [37]:
#снова копируем функцию из тренировочного ноутбука

def get_bootstrap_samples(data, n_samples):
    data = np.array(data)
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices.astype(int)]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [45]:
np.random.seed(0)
aspirin_ = np.array(odds(get_bootstrap_samples(aspirin_takers,1000)))
placebo_ = np.array(odds(get_bootstrap_samples(placebo_takers,1000)))

In [47]:
print('95% интервал:',str(stat_intervals(placebo_ / aspirin_, 0.05)))

ValueError: operands could not be broadcast together with shapes (11034,) (11037,) 